In [3]:
!pip install pytorch-lightning


^C
ERROR: Operation cancelled by user


In [4]:
import numpy as np
import pandas as pd 
import pytorch_lightning as L
from torchvision import transforms, models,datasets
import cv2
from torch.utils.data import Dataset, DataLoader ,random_split,Subset
import matplotlib.pyplot as plt 
import torch.nn as nn 
import torch.optim as optim 
from torchmetrics import MetricCollection, Accuracy
import torch.nn.functional as F
import torch
import os
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [5]:
os.environ['PATH_DATASETS'] = '/kaggle/input/neurolist/inaturalist_12K'


In [ ]:
wandb.login()
#760091de6b192857b226ee4bdecf4e7f93175087

sweep_config = {
  'name': 'Assignment2_partA',
  'method': 'bayes',
  'metric': {
      'name': 'Val Accuracy',
      'goal': 'maximize'   
    },
  'parameters': {
      'epochs': {
            'values': [5,10,15]
        },
        'conv_attributes_channels': {
            'values': [[32,64,32,64,32],[32,32,32,32,32],[16,32,64,128,256],[32,64,128,256,512],[256,128,64,32,16],[64,64,64,64,64]]
        },
        'conv_attributes_kernel_size': {
            'values': [[3,3,5,7,9],[7,5,5,3,3],[11,7,5,3,3],[3,3,3,5,5],[3,3,3,3,3],[11,7,7,5,3],[11,9,7,5,3],[3,5,7,9,11]]
        },
        'pool_attributes_kernel_size': {
            'values': [[2,2,2,2,2],[2,2,2,1,1],[2,1,3,1,2],[3,3,3,2,2]]
        },
        'pool_attributes_stride': {
            'values': [[2,2,2,2,2],[2,2,2,1,1],[1,1,2,2,2],[1,2,1,2,1],[2,2,2,2,1]]
        },
        'dense_layer_size': {
            'values': [32,64,128,256,512]
        },
        'learning_rate': {
            'values': [0.001,0.0015,0.0001, 0.01]
        },
        'activation': {
            'values': ['relu','elu','gelu','elu']
        },
        'dropout': {
            'values': [0.0 ,0.2 ,0.3 ,0.4 ,0.5]
        },
        'batch_normalization': {
            'values': [True,False]
        },
        'batch_size': {
            'values': [8,16, 32, 64]
        },
        'optimizer':{
              'values': ['adam','nadam','sgd']
        }
    }
}
sweep_id = wandb.sweep(sweep_config,entity="amar_cs23m011",project="Assignment2-CS6910")

In [6]:
class root_dataset(Dataset):
    def __init__(self):
        self.dataset1=datasets.ImageFolder(root='/kaggle/input/neurolist/inaturalist_12K/train')
        l1=int(len(self.dataset1)*0.8)
        train_dataset,val_dataset=random_split(self.dataset1, [int(len(self.dataset1)*0.8),len(self.dataset1)-l1])
        #print(len(train_dataset),len(val_dataset))
        self.train_dataset=train_dataset
        self.val_dataset=val_dataset
    def get_train_data(self):
        return self.train_dataset
    def get_val_data(self):
        return self.val_dataset

In [7]:
class inaturalist_train(Dataset):
    def __init__(self,train_data):
        self.target_size=(3,224,224)
        #dataset1=datasets.ImageFolder(root='/kaggle/input/neurolist/inaturalist_12K/train')
        self.dataset=train_data    
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),
            transforms.ToTensor()
        ])#self.target_size = target_size 
        
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [8]:
class inaturalist_val(Dataset):
    def __init__(self,val_data):
        self.target_size=(3,224,224)
        #dataset1=datasets.ImageFolder(root='/kaggle/input/neurolist/inaturalist_12K/train')
        self.dataset=val_data
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),
            transforms.ToTensor()
        ])#self.target_size = target_size   
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [9]:
class inaturalist_test(Dataset):
    def __init__(self):
        self.target_size=(3,224,224)
        self.dataset=datasets.ImageFolder(root='/kaggle/input/neurolist/inaturalist_12K/val')
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),  # Resize images to target size
            transforms.ToTensor()
        ])#self.target_size = target_size   
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [10]:
class Activation_Function:
    def activation_Function(self,activation_function):
        if activation_function=='relu':
            return F.relu
        if activation_function=='gelu':
            return F.gelu
        if activation_function=='selu':
            return F.selu
        if activation_function=='elu':
            return F.elu

In [11]:
dataset2=inaturalist_test()
print(len(dataset2))
dataloader=DataLoader(dataset=dataset2,batch_size=8,shuffle=False,num_workers=1)
datatiter=iter(dataloader)
feature,labels=next(datatiter)
print(feature.shape,labels)

2000
torch.Size([8, 3, 224, 224]) tensor([0, 0, 0, 0, 0, 0, 0, 0])


In [12]:
class Lightning_CNN(L.LightningModule):
    def __init__(self,layers,kernel_size,pool_kernel,pool_stride,dense_layer_size,batch_normalization,drop_out,a_fun,optimizer,dense_layer_output,learning_rate):
        super().__init__()
        self.batch_normalization=batch_normalization
        self.drop_out=drop_out
        self.optimizer=optimizer
        act_object=Activation_Function()
        self.dense_layer_output=dense_layer_output
        self.act_fun=act_object.activation_Function(a_fun)
        self.learning_rate=learning_rate
        self.conv1 = nn.Conv2d(3,layers[0], kernel_size=kernel_size[0], padding=1)
        self.b1=nn.BatchNorm2d(layers[0])
        self.pool1 = nn.MaxPool2d(kernel_size=pool_kernel[0], stride=pool_stride[0])
        self.conv2 = nn.Conv2d(layers[0],layers[1],kernel_size=kernel_size[1], padding=1)
        self.b2=nn.BatchNorm2d(layers[1])
        self.pool2 = nn.MaxPool2d(kernel_size=pool_kernel[1], stride=pool_stride[1])
        self.conv3 = nn.Conv2d(layers[1],layers[2], kernel_size=kernel_size[2], padding=1)
        self.b3=nn.BatchNorm2d(layers[2])
        self.pool3 = nn.MaxPool2d(kernel_size=pool_kernel[2], stride=pool_stride[2])
        self.conv4 = nn.Conv2d(layers[2],layers[3], kernel_size=kernel_size[3], padding=1)
        self.b4=nn.BatchNorm2d(layers[3])
        self.pool4 = nn.MaxPool2d(kernel_size=pool_kernel[3], stride=pool_stride[3])
        self.conv5 = nn.Conv2d(layers[3],layers[4], kernel_size=kernel_size[4], padding=1)
        self.b5=nn.BatchNorm2d(layers[4])
        self.pool5 = nn.MaxPool2d(kernel_size=pool_kernel[4], stride=pool_stride[4])
        self.dropout = nn.Dropout(p=drop_out)
        self.fc1 = nn.Linear(dense_layer_size, self.dense_layer_output)
        self.fc2 = nn.Linear(self.dense_layer_output, 10)
    def forward(self,x):
        if self.batch_normalization==True:
            x=self.pool1(self.act_fun(self.b1(self.conv1(x))))
            x=self.pool2(self.act_fun(self.b2(self.conv2(x))))
            x=self.pool3(self.act_fun(self.b3(self.conv3(x))))
            x=self.pool4(self.act_fun(self.b4(self.conv4(x))))
            x=self.pool5(self.act_fun(self.b5(self.conv5(x))))
        else:
            x=self.pool1(self.act_fun(self.conv1(x)))
            x=self.pool2(self.act_fun(self.conv2(x)))
            x=self.pool3(self.act_fun(self.conv3(x)))
            x=self.pool4(self.act_fun(self.conv4(x)))
            x=self.pool5(self.act_fun(self.conv5(x)))

        x=self.dropout(x)
        x=torch.flatten(x,1)
        x=self.dropout(x)
        x=self.act_fun(self.fc1(x))
        x=self.fc2(x)
        return x
    def training_step(self, batch, batch_idx):
        inputs,labels=batch
        output=self(inputs)
        _,preds = torch.max(output, dim=1)
        loss=F.cross_entropy(output,labels)
        #train_acc = torch.mean(preds == labels)
        #print(pred.shape)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    def configure_optimizers(self):
        if self.optimizer=='adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
            return optimizer
        if self.optimizer=='nadam':
            optimizer = torch.optim.NAdam(self.parameters(), lr=self.learning_rate)
            return optimizer
        if self.optimizer=='sgd':
            optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
            return optimizer
        
    def validation_step(self,batch,batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        val_loss = F.cross_entropy(y_pred, y)
        
        # Compute validation accuracy
        _, predicted = torch.max(y_pred, 1)
        val_acc = torch.sum(predicted == y).item() / y.size(0)
        
        self.log('val_loss', val_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', val_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return val_loss
        
    
    def test_step(self, batch, batch_idx):
        x,y=batch
        pred=self(x)
        loss=F.cross_entropy(pred,y)
        _, predicted = torch.max(pred, 1)
        accuracy = torch.sum(predicted == y).item() / y.size(0)
        #print(predicted,accuracy)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_accuracy", accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"test_loss": loss}
     


In [15]:
def main(config=None):
    with wandb.init(config=config, ):
        config=wandb.config
        wandb.run.name = 'bs-'+str(config.batch_size)+'-lr-'+ str(config.learning_rate)+'-ep-'+str(config.epochs)+ '-op-'+str(config.optimizer)+ '-dls-'+str(config.dense_layer_size)+ '-act-'+str(config.activation)+'-do-'+str(config.dropout)+'-bn-'+str(config.batch_normalization)+'-cs-'+str
        layers=config.conv_attributes_channels
        kernel_size=config.conv_attributes_kernel_size
        pool_kernel=config.pool_attributes_kernel_size
        pool_stride=config.pool_attributes_stride
        batch_normalization=config.batch_normalization
        drop_out=config.dropout
        activation_function=config.activation
        optimizer=config.optimizer
        b_size=config.batch_size
        dense_layer_output=config.dense_layer_size
        epoch=config.epochs
        learning_rate=config.learning_rate
    #aug_bit=True
        i_d=224
        D=0
        for i in range(5):
            D = (i_d - kernel_size[i])+3
            D = (D - pool_kernel[i])//pool_stride[i] + 1
            i_d = D
        root_obj=root_dataset()
        train_data=root_obj.get_train_data()
        val_data=root_obj.get_val_data()
        dataset1=inaturalist_train(train_data)
        dataset2=inaturalist_val(val_data)
        dataset3=inaturalist_test()
    #print(len(dataset1))
    #print(len(dataset2))
        dataloader=DataLoader(dataset=dataset1,batch_size=b_size,shuffle=True,num_workers=2)
        val_dataloader=DataLoader(dataset=dataset2,batch_size=b_size,shuffle=False,num_workers=2)
        model=Lightning_CNN(layers,kernel_size,pool_kernel,pool_stride,(D**2)*layers[4],batch_normalization,drop_out,activation_function,optimizer,dense_layer_output,learning_rate) 
        trainer = L.Trainer(accelerator='auto',devices="auto",max_epochs=epoch)
        trainer.fit(model,dataloader,val_dataloader)
        test_dataloader=DataLoader(dataset=dataset3,batch_size=8,shuffle=True,num_workers=1)
        trainer.test(dataloaders=test_dataloader)

In [16]:
if  __name__ =="__main__":
    wandb.agent(sweep_id, main, count=60)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.398499995470047     │
│         test_loss         │    1.7247939109802246     │
└───────────────────────────┴───────────────────────────┘